Lambda School Data Science

*Unit 2, Sprint 2, Module 3*

---

# Cross-Validation


## Assignment
- [ ] [Review requirements for your portfolio project](https://lambdaschool.github.io/ds/unit2), then submit your dataset.
- [ ] Continue to participate in our Kaggle challenge. 
- [ ] Use scikit-learn for hyperparameter optimization with RandomizedSearchCV.
- [ ] Submit your predictions to our Kaggle competition. (Go to our Kaggle InClass competition webpage. Use the blue **Submit Predictions** button to upload your CSV file. Or you can use the Kaggle API to submit your predictions.)
- [ ] Commit your notebook to your fork of the GitHub repo.


You won't be able to just copy from the lesson notebook to this assignment.

- Because the lesson was ***regression***, but the assignment is ***classification.***
- Because the lesson used [TargetEncoder](https://contrib.scikit-learn.org/categorical-encoding/targetencoder.html), which doesn't work as-is for _multi-class_ classification.

So you will have to adapt the example, which is good real-world practice.

1. Use a model for classification, such as [RandomForestClassifier](https://scikit-learn.org/stable/modules/generated/sklearn.ensemble.RandomForestClassifier.html)
2. Use hyperparameters that match the classifier, such as `randomforestclassifier__ ...`
3. Use a metric for classification, such as [`scoring='accuracy'`](https://scikit-learn.org/stable/modules/model_evaluation.html#common-cases-predefined-values)
4. If you’re doing a multi-class classification problem — such as whether a waterpump is functional, functional needs repair, or nonfunctional — then use a categorical encoding that works for multi-class classification, such as [OrdinalEncoder](https://contrib.scikit-learn.org/categorical-encoding/ordinal.html) (not [TargetEncoder](https://contrib.scikit-learn.org/categorical-encoding/targetencoder.html))



## Stretch Goals

### Reading
- Jake VanderPlas, [Python Data Science Handbook, Chapter 5.3](https://jakevdp.github.io/PythonDataScienceHandbook/05.03-hyperparameters-and-model-validation.html), Hyperparameters and Model Validation
- Jake VanderPlas, [Statistics for Hackers](https://speakerdeck.com/jakevdp/statistics-for-hackers?slide=107)
- Ron Zacharski, [A Programmer's Guide to Data Mining, Chapter 5](http://guidetodatamining.com/chapter5/), 10-fold cross validation
- Sebastian Raschka, [A Basic Pipeline and Grid Search Setup](https://github.com/rasbt/python-machine-learning-book/blob/master/code/bonus/svm_iris_pipeline_and_gridsearch.ipynb)
- Peter Worcester, [A Comparison of Grid Search and Randomized Search Using Scikit Learn](https://blog.usejournal.com/a-comparison-of-grid-search-and-randomized-search-using-scikit-learn-29823179bc85)

### Doing
- Add your own stretch goals!
- Try other [categorical encodings](https://contrib.scikit-learn.org/categorical-encoding/). See the previous assignment notebook for details.
- In additon to `RandomizedSearchCV`, scikit-learn has [`GridSearchCV`](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.GridSearchCV.html). Another library called scikit-optimize has [`BayesSearchCV`](https://scikit-optimize.github.io/notebooks/sklearn-gridsearchcv-replacement.html). Experiment with these alternatives.
- _[Introduction to Machine Learning with Python](http://shop.oreilly.com/product/0636920030515.do)_ discusses options for "Grid-Searching Which Model To Use" in Chapter 6:

> You can even go further in combining GridSearchCV and Pipeline: it is also possible to search over the actual steps being performed in the pipeline (say whether to use StandardScaler or MinMaxScaler). This leads to an even bigger search space and should be considered carefully. Trying all possible solutions is usually not a viable machine learning strategy. However, here is an example comparing a RandomForestClassifier and an SVC ...

The example is shown in [the accompanying notebook](https://github.com/amueller/introduction_to_ml_with_python/blob/master/06-algorithm-chains-and-pipelines.ipynb), code cells 35-37. Could you apply this concept to your own pipelines?


### BONUS: Stacking!

Here's some code you can use to "stack" multiple submissions, which is another form of ensembling:

```python
import pandas as pd

# Filenames of your submissions you want to ensemble
files = ['submission-01.csv', 'submission-02.csv', 'submission-03.csv']

target = 'status_group'
submissions = (pd.read_csv(file)[[target]] for file in files)
ensemble = pd.concat(submissions, axis='columns')
majority_vote = ensemble.mode(axis='columns')[0]

sample_submission = pd.read_csv('sample_submission.csv')
submission = sample_submission.copy()
submission[target] = majority_vote
submission.to_csv('my-ultimate-ensemble-submission.csv', index=False)
```

In [1]:
%%capture
import sys

# If you're on Colab:
if 'google.colab' in sys.modules:
    DATA_PATH = 'https://raw.githubusercontent.com/LambdaSchool/DS-Unit-2-Kaggle-Challenge/master/data/'
    !pip install category_encoders==2.*

# If you're working locally:
else:
    DATA_PATH = '../data/'

In [2]:
# Cross validation, so no val set

import pandas as pd

# Merge train_features.csv & train_labels.csv
train = pd.merge(pd.read_csv(DATA_PATH+'waterpumps/train_features.csv'), 
                 pd.read_csv(DATA_PATH+'waterpumps/train_labels.csv'))

# Read test_features.csv & sample_submission.csv
test = pd.read_csv(DATA_PATH+'waterpumps/test_features.csv')
sample_submission = pd.read_csv(DATA_PATH+'waterpumps/sample_submission.csv')

In [3]:
# Wrangle train and test in the same way

import numpy as np

def wrangle(X):
  # avoid copy warning
  X = X.copy()

  # replace incorrect latitudes with 0
  X['latitude'] = X['latitude'].replace(-2e-08, 0)
  X['construction_year'] = X['construction_year'].replace(0, np.NaN)

  # replace zeros with NaNs
  cols_with_zeros = ['latitude', 'longitude']
  for col in cols_with_zeros:
    X[col] = X[col].replace(0, np.nan)

  # drop a redundant column
  X = X.drop(columns='quantity_group')

  return X

train = wrangle(train)
test = wrangle(test)

In [5]:
target = 'status_group'


# Get a df with all features except the target and id
train_features = train.drop(columns=[target, 'id'])

# get a list of numeric features
numeric_features = train_features.select_dtypes(include='number').columns.tolist()

# get the cardinality of nonnumeric features
cardinality = train_features.select_dtypes(exclude='number').nunique()

# get features with cardinality <= 50
categorical_features = cardinality[cardinality <= 50].index.tolist()

# combine the two lists
features = numeric_features + categorical_features

In [6]:
X_train = train[features]
y_train = train[target]
X_test = test[features]

In [11]:
train.head()

,id,amount_tsh,date_recorded,funder,gps_height,installer,longitude,latitude,wpt_name,num_private,...,payment_type,water_quality,quality_group,quantity,source,source_type,source_class,waterpoint_type,waterpoint_type_group,status_group
0,69572,6000.0,2011-03-14,Roman,1390,Roman,34.938093,-9.856322,none,0,...,annually,soft,good,enough,spring,spring,groundwater,communal standpipe,communal standpipe,functional
1,8776,0.0,2013-03-06,Grumeti,1399,GRUMETI,34.698766,-2.147466,Zahanati,0,...,never pay,soft,good,insufficient,rainwater harvesting,rainwater harvesting,surface,communal standpipe,communal standpipe,functional
2,34310,25.0,2013-02-25,Lottery Club,686,World vision,37.460664,-3.821329,Kwa Mahundi,0,...,per bucket,soft,good,enough,dam,dam,surface,communal standpipe multiple,communal standpipe,functional
3,67743,0.0,2013-01-28,Unicef,263,UNICEF,38.486161,-11.155298,Zahanati Ya Nanyumbu,0,...,never pay,soft,good,dry,machine dbh,borehole,groundwater,communal standpipe multiple,communal standpipe,non functional
4,19728,0.0,2011-07-13,Action In A,0,Artisan,31.130847,-1.825359,Shuleni,0,...,never pay,soft,good,seasonal,rainwater harvesting,rainwater harvesting,surface,communal standpipe,communal standpipe,functional


In [16]:
# Do cross-validation on train

import category_encoders as ce
from sklearn.impute import SimpleImputer
from sklearn.model_selection import cross_val_score
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestClassifier

import category_encoders as ce
from sklearn.ensemble import RandomForestClassifier
from sklearn.impute import SimpleImputer
from sklearn.pipeline import make_pipeline

pipeline = make_pipeline(
    ce.OrdinalEncoder(), 
    SimpleImputer(strategy='median'), 
    RandomForestClassifier(random_state=0, n_jobs=-1)
)

k = 10
scores = cross_val_score(pipeline, X_train, y_train, cv=k, 
                         scoring='accuracy')
print(f'Accuracy for {k} folds:', scores)

Accuracy for 10 folds: [0.80925926 0.81515152 0.80808081 0.803367   0.82104377 0.7989899
 0.81094276 0.79579125 0.7993266  0.80841751]


In [17]:
# Do hyperparameter optimization with RandomizedSearchCV

from sklearn.model_selection import GridSearchCV, RandomizedSearchCV
from scipy.stats import randint, uniform

param_distributions = {
    'simpleimputer__strategy': ['mean', 'median', 'most_frequent'], 
    'randomforestclassifier__n_estimators': randint(50, 500), 
    'randomforestclassifier__max_depth': [5, 10, 15, 20, None], 
    'randomforestclassifier__max_features': uniform(0, 1), 
}

# If you're on Colab, decrease n_iter & cv parameters
search = RandomizedSearchCV(
    pipeline, 
    param_distributions=param_distributions, 
    n_iter=10, 
    cv=5, 
    scoring='accuracy', 
    verbose=10, 
    return_train_score=True, 
    n_jobs=-1
)

search.fit(X_train, y_train);

Fitting 5 folds for each of 10 candidates, totalling 50 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done   5 tasks      | elapsed:  1.7min
[Parallel(n_jobs=-1)]: Done  10 tasks      | elapsed:  4.9min
[Parallel(n_jobs=-1)]: Done  17 tasks      | elapsed:  7.5min
[Parallel(n_jobs=-1)]: Done  24 tasks      | elapsed:  9.1min
[Parallel(n_jobs=-1)]: Done  33 tasks      | elapsed: 12.1min
[Parallel(n_jobs=-1)]: Done  42 tasks      | elapsed: 15.6min
[Parallel(n_jobs=-1)]: Done  50 out of  50 | elapsed: 16.1min finished


In [18]:
# get best parameters

print('Best hyperparameters', search.best_params_)
print('Cross-validation accuracy', search.best_score_)

Best hyperparameters {'randomforestclassifier__max_depth': 20, 'randomforestclassifier__max_features': 0.8488968567566988, 'randomforestclassifier__n_estimators': 244, 'simpleimputer__strategy': 'most_frequent'}
Cross-validation accuracy 0.8075084175084175


In [19]:
# use best estimator to predict test data, then .to_csv

pipeline = search.best_estimator_

y_pred = pipeline.predict(X_test)

submission = sample_submission.copy()
submission['status_group'] = y_pred
submission.to_csv('isaacsubmission3.csv', index=False)

In [20]:
# Next I'm going to try ensembling my 3 submissions.
# Then, I'm going to try a basic logistic regression, and ensemble THAT on.
# My idea is that if there's systematic error, a different model type that
# measures something different might mitigate it.

In [22]:
# Filenames of your submissions you want to ensemble

files = ['dummysubmission1.csv', 'isaacsubmission2.csv', 'isaacsubmission3.csv']

target = 'status_group'
submissions = (pd.read_csv(file)[[target]] for file in files)
ensemble = pd.concat(submissions, axis='columns')
majority_vote = ensemble.mode(axis='columns')[0]


submission4 = sample_submission.copy()
submission4[target] = majority_vote
submission4.to_csv('isaac-first-3-ensemble-submission.csv', index=False)

In [24]:
from sklearn.ensemble import GradientBoostingClassifier

pipeline_boost = make_pipeline(
    ce.OrdinalEncoder(), 
    SimpleImputer(strategy='median'), 
    GradientBoostingClassifier(random_state=0)
)

k = 4
scores = cross_val_score(pipeline_boost, X_train, y_train, cv=k, 
                         scoring='accuracy')
print(f'Accuracy for {k} folds:', scores)

Accuracy for 4 folds: [0.74835017 0.75063973 0.74424242 0.75616162]


In [25]:
param_distributions_boost = {
    'simpleimputer__strategy': ['mean', 'median'], 
    'gradientboostingclassifier__n_estimators': randint(50, 500), 
    'gradientboostingclassifier__max_depth': [1, 2, 3, 4, None], 
    'gradientboostingclassifier__max_features': uniform(0, 1), 
}

# If you're on Colab, decrease n_iter & cv parameters
search_boost = RandomizedSearchCV(
    pipeline_boost, 
    param_distributions=param_distributions_boost, 
    n_iter=6, 
    cv=3, 
    scoring='accuracy', 
    verbose=10, 
    return_train_score=True, 
    n_jobs=-1
)

search_boost.fit(X_train, y_train);

Fitting 3 folds for each of 6 candidates, totalling 18 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done   5 tasks      | elapsed:  5.0min
[Parallel(n_jobs=-1)]: Done  10 tasks      | elapsed:  5.9min
[Parallel(n_jobs=-1)]: Done  13 out of  18 | elapsed:  8.0min remaining:  3.1min
[Parallel(n_jobs=-1)]: Done  15 out of  18 | elapsed:  8.6min remaining:  1.7min
[Parallel(n_jobs=-1)]: Done  18 out of  18 | elapsed:  9.4min finished


In [26]:
# get best parameters for GradientBoost model

print('Best hyperparameters', search_boost.best_params_)
print('Cross-validation accuracy', search_boost.best_score_)

Best hyperparameters {'gradientboostingclassifier__max_depth': None, 'gradientboostingclassifier__max_features': 0.47949208344580385, 'gradientboostingclassifier__n_estimators': 89, 'simpleimputer__strategy': 'median'}
Cross-validation accuracy 0.7959427609427608


In [27]:
# use best estimator to predict test data, then .to_csv

pipeline_boost = search_boost.best_estimator_

y_pred_boost = pipeline_boost.predict(X_test)

submission = sample_submission.copy()
submission['status_group'] = y_pred_boost
submission.to_csv('isaacsubmissionboost.csv', index=False)

In [28]:
# Create a 5-vote scheme, give my best model 2 votes since I only have 4 models

files = ['dummysubmission1.csv', 'isaacsubmission2.csv', 'isaacsubmission3.csv', 
         'isaacsubmission3.csv', 'isaacsubmissionboost.csv']

target = 'status_group'
submissions = (pd.read_csv(file)[[target]] for file in files)
ensemble = pd.concat(submissions, axis='columns')
majority_vote = ensemble.mode(axis='columns')[0]


submission5 = sample_submission.copy()
submission5[target] = majority_vote
submission5.to_csv('isaac-5-vote-ensemble-submission1.csv', index=False)

In [29]:
sample_submission

,id,status_group
0,50785,functional
1,51630,functional
2,17168,functional
3,45559,functional
4,49871,functional
...,...,...
14353,39307,functional
14354,18990,functional
14355,28749,functional
14356,33492,functional


In [30]:
sample_submission['status_group'].value_counts()

functional    14358
Name: status_group, dtype: int64

In [34]:
# saved various 3- and 5- vote schemes from here

files = ['dummysubmission1.csv', 'isaacsubmission2.csv', 'isaacsubmission3.csv', 
         DATA_PATH+'waterpumps/sample_submission.csv', 'isaacsubmissionboost.csv']

target = 'status_group'
submissions = (pd.read_csv(file)[[target]] for file in files)
ensemble = pd.concat(submissions, axis='columns')
majority_vote = ensemble.mode(axis='columns')[0]


submission6 = sample_submission.copy()
submission6[target] = majority_vote
submission6.to_csv('isaac-other-5-vote-ensemble-submission.csv', index=False)